In [1]:
import yaml
from sqlalchemy import create_engine
from sqlalchemy import inspect
import pandas as pd
import numpy as np
from data_cleaning import DataCleaning
from data_extraction import DataExtractor
import tabula

if __name__ == '__main__':
    database_extractor = DataExtractor()
    creds_dict = database_extractor.read_db_creds("db_creds.yaml")
    db_engine = database_extractor.init_db_engine(creds_dict)
    user_table = database_extractor.read_rds_table("legacy_users")

In [2]:
extractor = DataExtractor()
store_data = extractor.retrieve_stores_data("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/store_details/{}")

In [101]:
store_data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 441 entries, 0 to 450
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   index          441 non-null    int64 
 1   address        440 non-null    object
 2   longitude      440 non-null    object
 3   locality       440 non-null    object
 4   store_code     441 non-null    object
 5   staff_numbers  441 non-null    object
 6   opening_date   441 non-null    object
 7   store_type     441 non-null    object
 8   latitude       440 non-null    object
 9   country_code   441 non-null    object
 10  continent      441 non-null    object
dtypes: int64(1), object(10)
memory usage: 41.3+ KB


In [95]:
store_data["opening_date"] = pd.to_datetime(
    store_data["opening_date"], errors="coerce"
)
# reformat the date column to YYYY-MM-DD
store_data["opening_date"] = store_data[
    "opening_date"
].dt.strftime("%Y-%m-%d")

In [97]:
store_data.continent.unique()

array(['Europe', 'America'], dtype=object)

In [102]:
store_data.head(12)

,index,address,longitude,locality,store_code,staff_numbers,opening_date,store_type,latitude,country_code,continent
0,0,NaN,NaN,NaN,WEB-1388012W,325,2010-06-12,Web Portal,None,GB,Europe
1,1,"Flat 72W, Sally isle, East Deantown, E7B 8EB, ...",51.62907,High Wycombe,HI-9B97EE4E,34,1996-10-25,Local,-0.74934,GB,Europe
2,2,"Heckerstraße 4/5, 50491 Säckingen, Landshut",48.52961,Landshut,LA-0772C7B9,92,2013-04-12,Super Store,12.16179,DE,Europe
3,3,"5 Harrison tunnel, South Lydia, WC9 2BE, Westbury",51.26,Westbury,WE-1DE82CEE,69,2014-01-02,Super Store,-2.1875,GB,Europe
4,4,"Studio 6, Stephen landing, South Simon, B77 2W...",53.0233,Belper,BE-18074576,35,2019-09-09,Local,-1.48119,GB,Europe
5,5,"Flat 92u, Christian harbors, Port Charlotte, N...",53.38333,Gainsborough,GA-CAD01AC2,36,1995-05-15,Local,-0.76667,GB,Europe
6,6,"7 Gillian rue, West Robertside, PH4 8NY, Ruthe...",55.82885,Rutherglen,RU-C603E990,92,2001-01-04,Super Store,-4.21376,GB,Europe
7,7,"Lilija-Heß-Allee 660, 34566 Regensburg, Stuttgart",48.78232,Stuttgart,ST-229D997E,34,2000-06-01,Local,9.17702,DE,Europe
8,8,"510 Jill Mill, South Laura, FL 38723, Kaukauna",44.27804,Kaukauna,KA-FA7ED3B8,31,2022-09-05,Local,-88.27205,US,America
9,9,"3 Lee valleys, West Janetview, DY4M 2RL, Hartley",51.38673,Hartley,HA-974352FE,20,2004-09-11,Local,0.30367,GB,Europe


In [80]:
values = []
for name in store_data["staff_numbers"]:
    if pd.notnull(name):  # Check if the value is not NaN
        for letter in name:
            if letter in "qwertyuiopasdfghjklmnbvcxzQWERTYUIOPLKJHGFDSAZXCVBNM!#$%&'()*+,:;?@[\]^_`{|}~":
                values.append(name)
                break
print(values)
#indices = store_data[store_data["staff_numbers"].isin(values)].index
#store_data.drop(indices, inplace=True)

[]


In [79]:
store_data['staff_numbers'] = store_data['staff_numbers'].str.replace('[a-zA-Z]', '')

C:\Users\domin\AppData\Local\Temp\ipykernel_3584\2681074741.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  store_data['staff_numbers'] = store_data['staff_numbers'].str.replace('[a-zA-Z]', '')


In [47]:
store_data.drop(labels=[217, 405, 437], axis=0, inplace=True)

In [19]:
store_data[store_data["address"].str.contains("NULL", regex=False)]

ValueError: Cannot mask with non-boolean array containing NA / NaN values

In [3]:
extractor = DataExtractor()
number_of_stores = extractor.list_number_of_stores("https://aqj7u5id95.execute-api.eu-west-1.amazonaws.com/prod/number_stores")
number_of_stores  # prints the number of stores in the business


451

In [70]:
card_table["expiry_date"].unique()

array(['09/26', '10/23', '09/27', '11/23', '07/27', '10/28', '11/27',
       '11/31', '01/29', '02/32', '08/28', '09/32', '05/30', '06/28',
       '07/24', '07/31', '03/32', '12/30', '12/25', '06/24', '10/30',
       '04/32', '05/28', '08/27', '02/23', '08/26', '02/24', '03/29',
       '06/25', '07/30', '01/24', '07/23', '06/31', '03/24', '02/25',
       '01/23', '02/30', '03/27', '09/25', '07/28', '03/28', '06/29',
       '04/23', '03/23', '05/27', '04/28', '07/32', '10/24', '11/26',
       '02/29', '08/25', '12/22', '07/29', '07/25', '05/31', '04/24',
       '05/25', '06/32', '05/32', '12/27', '09/31', '01/31', '11/25',
       '12/28', '10/27', '03/26', '03/31', '10/25', '08/29', '01/28',
       '06/30', '05/26', '04/29', '08/30', '12/23', '11/29', '02/31',
       '09/29', '09/28', '01/27', '05/24', '08/31', '03/25', '04/31',
       '11/24', '08/23', '04/30', '02/28', '01/30', '12/26', '10/26',
       '04/25', '12/31', '03/30', '11/30', '08/24', '06/23', '06/27',
       '10/31', '09/

In [6]:
user_table.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15320 entries, 0 to 1249
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   first_name     15320 non-null  object
 1   last_name      15320 non-null  object
 2   date_of_birth  15320 non-null  object
 3   company        15320 non-null  object
 4   email_address  15320 non-null  object
 5   address        15320 non-null  object
 6   country        15320 non-null  object
 7   country_code   15320 non-null  object
 8   phone_number   15320 non-null  object
 9   join_date      15320 non-null  object
 10  user_uuid      15320 non-null  object
dtypes: object(11)
memory usage: 1.4+ MB


In [2]:
data_cleaner = DataCleaning()
user_table = data_cleaner.clean_user_data(user_table)

In [5]:
user_table.join_date.unique()

array(['2018-10-10', '2001-12-20', '2016-12-16', ..., '2016-04-15',
       '2021-03-07', '2015-08-28'], dtype=object)

In [16]:
#user_table.set_index("index", inplace=True)
user_table.sample(8)

,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid
index,,,,,,,,,,,
3405,Heinz-Willi,Schinke,2004-01-01,Gieß AG & Co. KGaA,karl-augustmatthaei@schueler.de,"Schuchhardtplatz 3/9, 58699 Bogen",Germany,DE,0049136585690,2002-04-01,5a57b0c9-7411-4ad6-806e-becbb1cc8318
6445,Edeltraut,Riehl,1979-08-13,Hartung Atzler GmbH,korbinian84@pruschke.de,"Gretel-Hentschel-Platz 3/0, 23554 Wernigerode",Germany,DE,00497590950165,2018-02-15,3ad79aa6-8c5f-493c-be47-0595eec2695a
7563,Birte,Höfig,1995-01-15,Schäfer GmbH & Co. KGaA,amielcarek@thies.net,"Gutknechtallee 532, 78970 Osterburg",Germany,DE,00493265418030,1996-02-09,8ed35d68-04b7-4b64-bb10-2f4d34aa8469
7888,Cathrin,Stiebitz,1983-04-13,Hiller,christlknappe@heidrich.net,"Laszlo-Etzold-Ring 5/5, 72362 Schongau",Germany,DE,0049998302017,2016-09-11,c7cd819a-bbff-49cd-80c9-483feb6041f1
6677,Joyce,Walsh,1973-09-07,Pearce-Wells,woodfiona@lee.org,"Flat 9, Miah summit, West Hayley, S4 3XW",United Kingdom,GB,00441914960737,2008-11-10,7bf9b15d-c8fd-470b-86be-2d7487e29c44
3404,Emilie,Stahr,1941-07-14,Holt,jdoehn@schweitzer.de,"Stjepan-Preiß-Allee 0/5, 60361 Gerolzhofen",Germany,DE,00499691167228,2005-10-20,5dd255d1-82bc-4628-8e8e-79ad1351dae5
14674,Debra,Townsend,1989-02-03,Wall-James,naomi47@brown.com,"5 Ian spur, Suttonton, HU9W 0RS",United Kingdom,GB,00442920180156,1999-02-07,d83a65aa-3e5e-448e-8d62-63068a24a307
10433,Larissa,Kraushaar,1962-12-09,Schlosser KG,gspiess@mueller.de,"Thiesstr. 15, 97488 Bützow",Germany,DE,0049366998195,2010-12-11,f7c5e6b6-43c8-40ea-8cbe-4e68c2d83f3f


In [36]:
user_table[user_table["phone_number"].str.contains("[qwertyuioplkjhgfdsazxcvbnm]", regex=True) & (user_table["join_date"] == "GB")]

,index,first_name,last_name,date_of_birth,company,email_address,address,country,country_code,phone_number,join_date,user_uuid


In [10]:
user_table.iloc[3187]

first_name                                             Chelsea
last_name                                               Murphy
date_of_birth                                       1940-04-28
company                                           Read-Harding
email_address                              jadepowell@bird.com
address          Flat 3, Preston curve, North Lindsey, ML9 6PU
country                                         United Kingdom
country_code                                                GB
phone_number                                    00449098790034
join_date                                           2017-07-17
user_uuid                 ad783032-0a1f-4632-8f33-b808d5918f88
Name: 3200, dtype: object

In [6]:
values = []
char = set()
for number in user_table["phone_number"]:
    for chars in number:
        if chars in "qwertyuioplkjhgfdsazxcvbnm!#$%&'()*.-+,/:;?@[\]^_`{|}~":
            values.append(number)
            char.add(chars)
            break
print(char)
#user_table[user_table['phone_number'].isin(values)]
#indices = user_table[user_table['first_name'].isin(values)].index
#user_table.drop(indices, inplace=True)

{'+', 'x', '('}


In [7]:
# check for nulls
user_table.isnull().sum()

first_name       0
last_name        0
date_of_birth    0
company          0
email_address    0
address          0
country          0
country_code     0
phone_number     0
join_date        0
user_uuid        0
dtype: int64